## Importing necessary libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import os
import tensorflow as tf

## Data loading

In [2]:
# data directories
val_folder = './chest_xray/val/'
test_folder = './chest_xray/test/'

In [3]:
# pre-defined load_image function

def load_images(folder_path):
    images = []
    labels = []

    for subfolder in os.listdir(folder_path):
        subfolder_path = os.path.join(folder_path, subfolder)
        if not os.path.isdir(subfolder_path):
            continue

        label = 'Normal' if subfolder == 'NORMAL' else 'Pneumonia'

        for filename in os.listdir(subfolder_path):
            file_path = os.path.join(subfolder_path, filename)
            img = cv2.imread(file_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            img = cv2.resize(img, (im_size, im_size))
            img = np.expand_dims(img, axis=-1) # adding back color channel
            images.append(img)
            labels.append(label)

    images = np.array(images)
    labels = np.array(labels)

    return images, labels

In [4]:
im_size = 224
val_images, val_labels = load_images(val_folder)
test_images, test_labels = load_images(test_folder)

In [5]:
train_images = np.load('train_images_balanced.npy')
train_labels = np.load('train_labels_balanced.npy')

In [6]:
train_images.shape

(7750, 224, 224, 1)

## Data encoding

Since this is a binary classification problem we can use Label Indexing.

In [7]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

train_labels_encoded = encoder.fit_transform(train_labels)
test_labels_encoded = encoder.fit_transform(test_labels)
val_labels_encoded = encoder.fit_transform(val_labels)

In [8]:
test_labels_encoded.shape

(624,)

In [32]:
print(np.asarray(np.unique(train_labels_encoded, return_counts=True)).T)

[[   0 3875]
 [   1 3875]]


## Building the Model

In [9]:
from tensorflow.keras import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense,Input,Conv2D,Flatten,MaxPool2D

In [10]:
# variables
input_shape = (im_size,im_size,1)
filter1_size = 32
filter2_size = 64
filter_shape = (3,3)
pool_shape = (2,2)

# Model architecture implementation
model = Sequential()
model.add(Conv2D(32, filter_shape, activation='relu', input_shape=input_shape))
# model.add(Conv2D(64, filter_shape, activation='relu'))
model.add(MaxPool2D(pool_shape))

# model.add(Conv2D(32, filter_shape, activation='relu'))
model.add(Conv2D(64, filter_shape, activation='relu'))
model.add(MaxPool2D(pool_shape))


#Flatten layer
model.add(Flatten())

model.add(Dense(128, activation='relu'))

NEXT TRY OUTPUT to 1 layer using sigmoid activation

In [11]:
# Dense (output) layer
output_shape = model.add(Dense(1, activation='sigmoid'))

# Create model class
model = Model(model.input, model.output)

# Print model summary
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_input (InputLayer)   [(None, 224, 224, 1)]     0         
                                                                 
 conv2d (Conv2D)             (None, 222, 222, 32)      320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 64)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 186624)            0     

Trainable params: 23,906,945
Non-trainable params: 0
_________________________________________________________________


In [12]:
X_train = train_images / 255.0

In [13]:
X_test = test_images / 255.0

## Training the Model

In [33]:
#epochs for model training and learning rate for optimizer
epochs = 20
learning_rate = 1e-2

#using Adam optimizer to compile or build the model
optimizer = Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer,
      loss='categorical_crossentropy',
      metrics=["accuracy"])

In [15]:
type(X_train)

numpy.ndarray

In [18]:
from keras.preprocessing.image import ImageDataGenerator

data_generator = ImageDataGenerator(rescale=1./255)

# Set the batch size and image size
batch_size = 32


train_dir = './chest_xray/train'
# Create the data generators
train_generator = data_generator.flow_from_directory(
    train_dir,
    batch_size=batch_size
)

Found 5216 images belonging to 2 classes.


In [34]:
hist = model.fit(X_train, train_labels_encoded, epochs=10, batch_size=32, validation_data=(X_test, test_labels_encoded))

Epoch 1/10
243/243 [==============================] - 16s 60ms/step - loss: 0.0000e+00 - accuracy: 0.5008 - val_loss: 0.0000e+00 - val_accuracy: 0.3750
Epoch 2/10
243/243 [==============================] - 14s 58ms/step - loss: 0.0000e+00 - accuracy: 0.5000 - val_loss: 0.0000e+00 - val_accuracy: 0.3750
Epoch 3/10
241/243 [============================>.] - ETA: 0s - loss: 0.0000e+00 - accuracy: 0.4995

: 

: 

In [ ]:
#Accuracy Graph
plt.figure(figsize=(8,6))
plt.title('Accuracy scores')
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.legend(['acc', 'val_acc'])
plt.show()

In [ ]:
#Loss Graph
plt.figure(figsize=(8,6))
plt.title('Loss value')
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.legend(['loss', 'val_loss'])
plt.show()

## Evaluating the model's performance

In [ ]:
labels = ["Normal", "Pneumonia"]

#confusion matrix
from sklearn.metrics import confusion_matrix
y_pred = model.predict(test_images)

#transforming label back to original
y_pred = encoder.inverse_transform(y_pred)
#matrix of Actual vs Prediction data
c_matrix = confusion_matrix(test_labels, y_pred)
plt.figure(figsize=(8,8))
plt.title('Confusion matrix',fontsize=14)
sns.heatmap(
  c_matrix, xticklabels=labels,yticklabels=labels,
  fmt='d', annot=True,annot_kws={"size": 14}, cmap='Reds')
plt.xlabel("Predicted",fontsize=12)
plt.ylabel("Actual",fontsize=12)
plt.show()

## Prediction

In [ ]:
for i in range(230,280,3):
    pred = model.predict(test_images)[i]
    #Display the x-ray image
    cv2.imshow("X-Ray",test_images[i])
    print("Actual :",test_labels[i]," Predicted :",labels[np.argmax(pred)])
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

accuracy = accuracy_score(test_labels, y_pred)
precision = precision_score(test_labels, y_pred)
recall = recall_score(test_labels, y_pred)
f1_score = f1_score(test_labels, y_pred)

print("Accuracy: ", accuracy)
print("Precision: ", precision)
print("Recall: ", recall)
print("F1 Score: ", f1_score)